In [2]:
import pandas as pd 
from scipy.stats import norm
import numpy as np

In [3]:
pip install xlrd

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\Phillip\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [24]:
returns_df = pd.read_excel('Problem Set 1 Data.xls', index_col = 0) + 1 


In [27]:
def portfolio_replication(
        returns,
        S_0 = 100,
        K = 90,
        r = 0.0,
        vol= 0.25,
        Total_Weeks = 20,
        asset = 'Return series 1'
):
    if isinstance(returns, pd.DataFrame):
        weekly_returns = returns[asset].copy()
    else:
        weekly_returns = returns.copy()
    
    #Create initial conditions for price process
    initial = pd.Series([1.0], index = [-1]) 
    #Use ignore_index to get around indexing issues. 
    all_returns = pd.concat([initial, weekly_returns], ignore_index = True)
    
    #Convert returns to stock price - starting from Week 0 - use .iloc to get rid of index = -1 item
    prices = (all_returns.cumprod() * S_0).iloc[1:]

    def BSM(S, tau, K = K, vol = vol, r = r):

        #Put value is just payoff at maturity
        if tau <= 0:
            put_price = np.max(K - S, 0)
            return put_price, 0 
        
        d1 = (np.log(S / K) + (r + 0.5 * vol**2 )*(tau)) / (vol * np.sqrt(tau))
        d2 = d1 - vol * np.sqrt(tau)
        put_price = K * np.exp( r * tau) * norm.cdf(-d2) - S * norm.cdf(-d1)
        delta = norm.cdf(d1) - 1 

        return put_price, delta
        
    
    ''' Creating Dataframe to store replicating portfolio values'''

    columns = ['Week', 'S_t', 'Time-2-Mat', 'BSM Price', 'Delta'
               , 'dDelta', 'Hedge', 'Cash Bal', 'Port Val']
    
    df = pd.DataFrame(index = np.arange(0, Total_Weeks + 1), columns = columns)

    ''' Creating the initial portfolio 
        - Cash position = Put Option - Delta * S 
        - Cash position is from shorting - Delta number of Shares 

    '''
    tau0 = Total_Weeks/52 
    P0, delta0 = BSM(S_0, tau = tau0)
    #Cash Position at t = 0
    cash0 = P0 - delta0 * S_0 

    df.loc[0] = [
        0, #Week
        S_0, #S_t
        tau0, #Time2Mat
        P0, #BSM Price
        delta0, #Delta
        np.nan, #dDelta 
        np.nan, #rebalancing trade 
        cash0, #cash position
        cash0 + delta0*S_0, #port val 
                ]
    
    ''' Step Forward : For Loop Replication '''
    Delta_prior_i = delta0
    cash_prior = cash0
    for period in range(1, Total_Weeks + 1):
        #Share Price in Period i 
        S_i = prices.loc[period]
        #New time to maturity
        tau_i = (Total_Weeks - period)/ 52
        #New Put Price
        Put_i, Delta_i = BSM(S_i, tau_i)
        #New Delta
        dDelta = Delta_i - Delta_prior_i
        #Hedge Trade
        hedge = -dDelta * S_i 
        #New Cash Balance
        cash_i = cash_prior + hedge 
        #Port Val
        port_val_i = cash_i + Delta_i * S_i 

        #Save to dataframe
        df.loc[period] = [period, S_i, tau_i, Put_i, Delta_i, dDelta, hedge, cash_i, port_val_i]

        #Reset Values
        Delta_prior_i = Delta_i
        cash_prior = cash_i 
    

    return df


df = portfolio_replication(returns = returns_df)
df2 = portfolio_replication(returns = returns_df, asset = 'Return series 2')

In [29]:
df

,Week,S_t,Time-2-Mat,BSM Price,Delta,dDelta,Hedge,Cash Bal,Port Val
0,0,100,0.384615,2.171644,-0.224502,NaN,NaN,24.621842,2.171644
1,1,91.499655,0.365385,4.748612,-0.426648,-0.202146,18.496305,43.118147,4.079988
2,2,88.283613,0.346154,6.128892,-0.522873,-0.096225,8.495105,51.613252,5.452106
3,3,85.742017,0.326923,7.420173,-0.605493,-0.08262,7.083991,58.697244,6.781039
4,4,88.233645,0.307692,5.859624,-0.529334,0.076159,-6.719793,51.97745,5.272375
5,5,88.209642,0.288462,5.717861,-0.53288,-0.003546,0.312793,52.290243,5.285081
6,6,89.557747,0.269231,4.867292,-0.489276,0.043604,-3.905093,48.38515,4.566703
7,7,87.925556,0.25,5.547532,-0.549363,-0.060088,5.283233,53.668383,5.365294
8,8,86.937508,0.230769,5.941933,-0.590263,-0.0409,3.555717,57.2241,5.908092
9,9,86.822846,0.211538,5.838087,-0.600667,-0.010404,0.90331,58.12741,5.975772


In [30]:
df2

,Week,S_t,Time-2-Mat,BSM Price,Delta,dDelta,Hedge,Cash Bal,Port Val
0,0,100,0.384615,2.171644,-0.224502,NaN,NaN,24.621842,2.171644
1,1,94.3457,0.365385,3.646257,-0.349153,-0.124651,11.76032,36.382161,3.441045
2,2,92.139898,0.346154,4.337019,-0.407764,-0.05861,5.400337,41.782499,4.211208
3,3,90.375418,0.326923,4.953134,-0.459937,-0.052173,4.715198,46.497696,4.930699
4,4,92.122375,0.307692,4.043405,-0.406167,0.05377,-4.953389,41.544307,4.127209
5,5,92.105705,0.288462,3.892882,-0.405406,0.000761,-0.070095,41.474212,4.13398
6,6,93.042051,0.269231,3.366217,-0.374059,0.031347,-2.916576,38.557636,3.754379
7,7,91.914103,0.25,3.639828,-0.408712,-0.034653,3.185096,41.742732,4.176299
8,8,91.227054,0.230769,3.752808,-0.431402,-0.022689,2.069892,43.812623,4.457104
9,9,91.14702,0.211538,3.604095,-0.433437,-0.002035,0.185496,43.998119,4.491631


# Question A 

BSM Price of Put is $2.171643

In [ ]:
df['BSM Price'][0]

np.float64(2.1716436485757598)

# Question B & C 

Trader's final cash balance is $21.16 dollars which is close to the Black-Sholes price of the put option at $21.57. This shows for this given returns series, the portfolio insurance strategy approximates a put option well. Additionally, the investor paid less to synethetically replicate a put option than outright buying the put. 

Smaller market movements (shown with a lower realised volatility) is beneficial for the vega-exposure that this strategy holds. In this return series, the synthethic put was replicated to be cheaper than the put premium

In [21]:
df.loc[20]

Week                 20
S_t           68.427302
Time-2-Mat          0.0
BSM Price     21.572698
Delta                 0
dDelta              1.0
Hedge        -68.427302
Cash Bal      21.163712
Port Val      21.163712
Name: 20, dtype: object

In [ ]:
(df.loc[20, 'Cash Bal'] - df.loc[20 , 'BSM Price']).round(2)

np.float64(-0.41)

# Question D

Final cash balance is $13.23 dollars. Compared to payoff of only holding the put would yield $12.03 dollars which is $1.20 less.

Realised volatility is 39.5% whereas implied volatility given by the question is 25% (the vol value used to price BSM-model puts). This volatility premium means the put option is 'expensive' as the holder of the protfolio could have. This intuitively makes sense, given implied volatility is essientally how traders see the price as options. Therefore, if implied volatility is higher than realised volatility, then the prices quoted by market makers is overvalued


In [32]:
df2.loc[20]

Week                 20
S_t            77.96968
Time-2-Mat          0.0
BSM Price      12.03032
Delta                 0
dDelta         0.999639
Hedge        -77.941542
Cash Bal       13.23482
Port Val       13.23482
Name: 20, dtype: object

In [ ]:
df2['S_t'].std() * np.sqrt(52)

np.float64(39.47957887722939)